### Построение модели на RandomForest

In [11]:
import org.apache.spark.sql.types.{StructType, StructField, IntegerType, LongType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.feature.{IndexToString, StringIndexer, VectorIndexer, HashingTF, Tokenizer}

import org.apache.spark.sql.types.{StructType, StructField, IntegerType, LongType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.feature.{IndexToString, StringIndexer, VectorIndexer, HashingTF, Tokenizer}


In [12]:
val dataSchema = new StructType()
    .add("target", IntegerType)
    .add("id", LongType)
    .add("raw_timestamp", StringType)
    .add("query_status", StringType)
    .add("author", StringType)
    .add("tweet", StringType)

    
val dataPath= "./data/training.1600000.processed.noemoticon.csv"

val raw_sentiment = spark.read
    .format("csv")
    .option("header",false)
    .schema(dataSchema)
    .load(dataPath)
    .selectExpr("(case when target=4 then 1 else 0 end) as label","tweet")

raw_sentiment.groupBy($"label").count.show

+-----+------+
|label| count|
+-----+------+
|    1|800000|
|    0|800000|
+-----+------+



dataSchema: org.apache.spark.sql.types.StructType = StructType(StructField(target,IntegerType,true), StructField(id,LongType,true), StructField(raw_timestamp,StringType,true), StructField(query_status,StringType,true), StructField(author,StringType,true), StructField(tweet,StringType,true))
dataPath: String = ./data/training.1600000.processed.noemoticon.csv
raw_sentiment: org.apache.spark.sql.DataFrame = [label: int, tweet: string]


In [13]:
val tokenizer = new Tokenizer()
    .setInputCol("tweet")
    .setOutputCol("words")

val hashingTF = new HashingTF()
    .setNumFeatures(1000)
    .setInputCol(tokenizer.getOutputCol)
    .setOutputCol("features")

// Index labels, adding metadata to the label column.
// Fit on whole dataset to include all labels in index.
val labelIndexer = new StringIndexer()
  .setInputCol("label")
  .setOutputCol("indexedLabel")
  .fit(raw_sentiment)

// Automatically identify categorical features, and index them.
// Set maxCategories so features with > 4 distinct values are treated as continuous.
val featureIndexer = new VectorIndexer()
  .setInputCol("features")
  .setOutputCol("indexedFeatures")
  .setMaxCategories(4)

// Train a RandomForest model.
val rf = new RandomForestClassifier()
  .setLabelCol("indexedLabel")
  .setFeaturesCol("indexedFeatures")
  .setNumTrees(10)

// Convert indexed labels back to original labels.
val labelConverter = new IndexToString()
  .setInputCol("prediction")
  .setOutputCol("predictedLabel")
  .setLabels(labelIndexer.labels)

// Chain indexers and forest in a Pipeline.
val pipeline = new Pipeline()
  .setStages(Array(tokenizer, hashingTF, labelIndexer, featureIndexer, rf, labelConverter))

tokenizer: org.apache.spark.ml.feature.Tokenizer = tok_6627fe32aacd
hashingTF: org.apache.spark.ml.feature.HashingTF = hashingTF_9d1c52c5bd0b
labelIndexer: org.apache.spark.ml.feature.StringIndexerModel = strIdx_5e6da8687587
featureIndexer: org.apache.spark.ml.feature.VectorIndexer = vecIdx_054e9a6ef315
rf: org.apache.spark.ml.classification.RandomForestClassifier = rfc_47053fa34d81
labelConverter: org.apache.spark.ml.feature.IndexToString = idxToStr_14b1daab331b
pipeline: org.apache.spark.ml.Pipeline = pipeline_17a6fd2fa90f


In [14]:
val model = pipeline.fit(raw_sentiment)

2020-02-24 14:06:59 WARN  MemoryStore:66 - Not enough space to cache rdd_196_6 in memory! (computed 46.0 MB so far)
2020-02-24 14:06:59 WARN  BlockManager:66 - Persisting block rdd_196_6 to disk instead.
2020-02-24 14:06:59 WARN  MemoryStore:66 - Not enough space to cache rdd_196_0 in memory! (computed 46.0 MB so far)
2020-02-24 14:06:59 WARN  BlockManager:66 - Persisting block rdd_196_0 to disk instead.
2020-02-24 14:06:59 WARN  MemoryStore:66 - Not enough space to cache rdd_196_7 in memory! (computed 46.0 MB so far)
2020-02-24 14:06:59 WARN  BlockManager:66 - Persisting block rdd_196_7 to disk instead.
2020-02-24 14:06:59 WARN  MemoryStore:66 - Not enough space to cache rdd_196_1 in memory! (computed 46.0 MB so far)
2020-02-24 14:06:59 WARN  BlockManager:66 - Persisting block rdd_196_1 to disk instead.
2020-02-24 14:06:59 WARN  MemoryStore:66 - Not enough space to cache rdd_196_3 in memory! (computed 46.0 MB so far)
2020-02-24 14:06:59 WARN  BlockManager:66 - Persisting block rdd_196

model: org.apache.spark.ml.PipelineModel = pipeline_17a6fd2fa90f


In [15]:
model.write.overwrite().save("./models/spark-ml-model-rf")

2020-02-24 14:07:54 WARN  MemoryManager:115 - Total allocation exceeds 95,00% (950 822 490 bytes) of heap memory
Scaling row group sizes to 88,55% for 8 writers


In [16]:
val sameModel = PipelineModel.load("./models/spark-ml-model-rf")

sameModel: org.apache.spark.ml.PipelineModel = pipeline_17a6fd2fa90f


In [17]:
val predictionsDF = sameModel.transform(raw_sentiment)

predictionsDF.show()

+-----+--------------------+--------------------+--------------------+------------+--------------------+--------------------+--------------------+----------+--------------+
|label|               tweet|               words|            features|indexedLabel|     indexedFeatures|       rawPrediction|         probability|prediction|predictedLabel|
+-----+--------------------+--------------------+--------------------+------------+--------------------+--------------------+--------------------+----------+--------------+
|    0|@switchfoot http:...|[@switchfoot, htt...|(1000,[7,14,21,54...|         0.0|(1000,[7,14,21,54...|[4.39665546729450...|[0.43966554672945...|       1.0|             1|
|    0|is upset that he ...|[is, upset, that,...|(1000,[170,193,22...|         0.0|(1000,[170,193,22...|[5.24037086071899...|[0.52403708607189...|       0.0|             0|
|    0|@Kenichan I dived...|[@kenichan, i, di...|(1000,[10,36,77,1...|         0.0|(1000,[10,36,77,1...|[5.89570496145148...|[0.5895704

predictionsDF: org.apache.spark.sql.DataFrame = [label: int, tweet: string ... 8 more fields]
